<a href="https://colab.research.google.com/github/princessivy/course/blob/main/Preprocessing_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installs & Imports

In [1]:
!pip install ndjson --quiet
!pip install beautifulsoup4 --quiet
!pip install html2text --quiet
!pip install nltk --quiet
!pip install HanTa --quiet
!pip install langdetect --quiet

     |████████████████████████████████| 1.5 MB 14.0 MB/s 
     |████████████████████████████████| 981 kB 14.9 MB/s 


In [2]:
import ndjson
import requests
import pandas as pd
from bs4 import BeautifulSoup
import gzip
from pathlib import Path
from google.colab import drive
from collections import Counter, OrderedDict
from google.colab import drive
import html2text
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction import text
from HanTa import HanoverTagger as ht
from langdetect import detect
import gc
gc.enable()

#Test-Dataset

In [ ]:
# Mount Google Drive
drive.mount('/gdrive')
data_path = Path('/gdrive/MyDrive/industry_data/')
file_name = 'test_small.ndjson.gz'

with gzip.open(data_path/file_name, "rt", encoding='UTF-8') as file:
    data = ndjson.load(file)
df_test = pd.DataFrame(data)

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
# check for null entries
if df_test.isnull().any(axis=None):
    print("\nPreview of data with null values:\nxxxxxxxxxxxxx")
    print(df_est[df_test.isnull().any(axis=1)].head(3))
    missingno.matrix(df_test)
    plt.show()
else:
  print('No null entries found')

No null entries found


In [ ]:
# generate count statistics of duplicate entries
if len(df_test[df_test.duplicated()]) > 0:
    print("No. of duplicated entries: ", len(df_test[df_test.duplicated()]))
    print(df_test[df_test.duplicated(keep=False)].sort_values(by=list(df_test.columns)).head())
else:
    print("No duplicated entries found")

No duplicated entries found


#Train-Dataset

In [3]:
drive.mount('/gdrive')
data_path = Path('/gdrive/MyDrive/industry_data/')
file_name = 'train_small.ndjson.gz'
with gzip.open(data_path/file_name, "rt", encoding='UTF-8') as file:
    data = []
    data = [ndjson.loads(line) for line in file]

Mounted at /gdrive


In [4]:
# Nested List rausholen, Flat-List erzeugen, um Daten in DataFrame zu bekommen
flat_list = [item for sublist in data for item in sublist]
df_train = pd.DataFrame(flat_list)

In [ ]:
# check for null entries
if df_train.isnull().any(axis=None):
    print("\nPreview of data with null values:\nxxxxxxxxxxxxx")
    print(df_train[df_train.isnull().any(axis=1)].head(3))
    missingno.matrix(df_train)
    plt.show()
else:
  print('No null entries found')

No null entries found


In [ ]:
# generate count statistics of duplicate entries
if len(df_train[df_train.duplicated()]) > 0:
    print("No. of duplicated entries: ", len(df_train[df_train.duplicated()]))
    print(df_train[df_train.duplicated(keep=False)].sort_values(by=list(df_train.columns)).head())
else:
    print("No duplicated entries found")

In [ ]:
gc.collect()

310

#HTML Feature-Checkout

##Checking out Tag-Occurence

In [ ]:
all_tags = [] #der ersten 1000 Datensätze

for i in range(1000): #len(df_train)
  soup = BeautifulSoup(data[i][0]['html'], 'html.parser')
  #for tag in soup.findAll(True):
    #print(tag.name)
  tags = set(tag.name for tag in BeautifulSoup(data[i][0]['html'], 'html.parser').find_all()) # oder direkt hier set (um zu schauen, ob es überhaupt in allen Datensätzen vorkommt; wenn später <1000 dann nämlich nicht)
  all_tags.extend(tags)

  #print(data[i][0]['url'])
  #print(soup.get_text()[:1024])
  #print(tags)

In [ ]:
# oder all_tags_set
counted = Counter(all_tags)
OrderedDict(counted.most_common())

##Whole Text between Tags

In [ ]:
pd.options.mode.chained_assignment = None

In [ ]:
def parse_to_text(html):
    soup = BeautifulSoup(html, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)

    return text

In [ ]:
# eliminate htlm elements from text

df_train = df_train.assign(html_to_text='')
for line in range(0, len(df_train)):
  #print(df_train.iloc[line]) #debugging
  #content = parse_html(df_train.html[line])
  content = parse_to_text(df_train.html[line])
  df_train.html_to_text[line] = content

In [63]:
df_train_work = df_train.head(100).copy() #----------------------------------------

In [64]:
df_train_work

,url,html,industry,industry_label
0,http://www.kaiser-personal-service.de,"<!doctype html>\n<html lang=""de"">\n\t<head>\n\...",137,Human Resources
1,http://www.brandseven.com,"<!DOCTYPE html>\n<html class=""html"" lang=""de-D...",96,Information Technology and Services
2,http://www.beumergroup.com,"<!DOCTYPE html>\n<html class=""home page-templa...",135,Mechanical or Industrial Engineering
3,http://www.aurigavision.com,<!doctype html>\n<!--[if IE 8]> <html ...,13,Medical Practice
4,http://www.comme-a-la-maison.ch,"<!DOCTYPE html>\n<html lang=""fr"">\n<head>\n\n<...",48,Construction
...,...,...,...,...
95,http://www.i-soft-systemhaus.de,"<!DOCTYPE html>\n\n<html lang=""de"">\n\n<head>\...",96,Information Technology and Services
96,http://www.communicode.de,"<!DOCTYPE html><html lang=""de""><head><script>(...",96,Information Technology and Services
97,http://www.kuper.de,"<!DOCTYPE html><html lang=""de""><head><script i...",135,Mechanical or Industrial Engineering
98,http://craftworks.at,"<!DOCTYPE html>\n<html lang=""en"">\n<head>\n \...",96,Information Technology and Services


###Preprocessing

In [28]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [29]:
# https://medium.com/analytics-vidhya/applying-text-classification-using-logistic-regression-a-comparison-between-bow-and-tf-idf-1f1ed1b83640

# hier könnten wir sprachenabhängig arbeiten: clean_text_german(), clean_text_english() und anhand des lang-tags anwenden
# 
def clean_text(mixed_text):
    '''Text Preprocessing '''
    
    # convert words to lower case
    content = mixed_text.lower()
    
    # ENGLISH use this for english text
    # Expand contractions (you've -> you have)
    #if True:
    #    text = text.split()
    #    new_text = []
    #    for word in text:
    #        if word in contractions:
    #            new_text.append(contractions[word])
    #        else:
    #            new_text.append(word)
    #    text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    #content = re.sub(r'https?:\/\/.*[\r\n]*', '', content, flags=re.MULTILINE) # brauchen wir nicht mehr, da schon geparst
    #content = re.sub(r'\<a href', ' ', content)
    content = re.sub(r'&amp;', '', content) 
    content = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', content)
    content = re.sub(r'<br />', ' ', content)
    content = re.sub(r'\'', ' ', content)
    
    # remove stopwords
    content = content.split()
    stops = set(stopwords.words('german'))
    content = [w for w in content if not w in stops]
    content = ' '.join(content)

    # tokenize each word
    content =  nltk.WordPunctTokenizer().tokenize(content)
    
    # lemmatize each token in German (reduce words to stem)
    tagger = ht.HanoverTagger('morphmodel_ger.pgz')
    word_list = []
    for w in content:
        lemma = [lemma for (word,lemma,pos) in tagger.tag_sent(w.split())]
        word_list.append(' '.join(lemma))

    # ENGLISH use this for english text
    # lemmatize each token
    #lemm = nltk.stem.WordNetLemmatizer()
    #content = list(map(lambda word:list(map(lemm.lemmatize, word)), content))
    
    return word_list

In [7]:
#df_train = df_train.assign(html_cleaned='')
df_train_work = df_train_work.assign(html_cleaned='')

In [9]:
for line in range(0, len(df_train_work)):
    # print(line) #debugging
    content = clean_text(df_train_work['html_to_text'][line])
    df_train_work.html_cleaned[line] = content

KeyError: ignored

##Specific Tags for additional Features

In [102]:
def getHTML(url):
    # später anpassen, wenn wir live-url abfragen! evtl. Fallunterscheidung?!
    '''    r = requests.get(url)
    r.text'''
    return  BeautifulSoup(url, 'html.parser')



## Img-Description from IMG-Tag
def getImgDescriptionHTMLtag(url):
    soup = getHTML(url)

    results = soup.find_all('img', alt = True)
    img_description = []
    for x in range(0,len(results)):
      first_result = results[x]
      img_description.append(first_result['alt'])
    
    return list(filter(None, img_description))


## Title
def getTitleHTMLtag(url):
    soup = getHTML(url)

    if (soup.title is not None):
        return str(soup.title.string)
    else:
        return ""


## h1
def getH1HTMLtag(url):
    soup = getHTML(url)

    heading = soup.findAll('h1')
    n = len(heading)

    liste = []
    for x in range(n):
      liste.append(str.strip(heading[x].text))

    return list(filter(None, liste))


## h2
def getH2HTMLtag(url):
    soup = getHTML(url)

    heading = soup.findAll('h2')
    n = len(heading)

    liste = []
    for x in range(n):
      liste.append(str.strip(heading[x].text))

    return list(filter(None, liste))


## h3
def getH3HTMLtag(url):
    soup = getHTML(url)

    heading = soup.findAll('h3')
    n = len(heading)

    liste = []
    for x in range(n):
      liste.append(str.strip(heading[x].text))

    return list(filter(None, liste))


## strong - fragwürdig
def getStrongHTMLtag(url):
    soup = getHTML(url)

    heading = soup.findAll('strong')
    n = len(heading)

    liste = []
    for x in range(n):
      liste.append(str.strip(heading[x].text))

    return list(filter(None, liste))


## bold
def getBoldHTMLtag(url):
    soup = getHTML(url)

    heading = soup.findAll('bold')
    n = len(heading)

    liste = []
    for x in range(n):
      liste.append(str.strip(heading[x].text))

    return list(filter(None, liste))
  

## language code
def getLangHTMLtag(url):
    soup = getHTML(url)

    body_text = soup.body.get_text()
    return detect(body_text)
    
    
## figcaption
def getFigCaptionHTMLtag(url):
    soup = getHTML(url)

    heading = soup.findAll('figcaption')
    n = len(heading)

    liste = []
    for x in range(n):
      liste.append(str.strip(heading[x].text))

    return list(filter(None, liste))

In [65]:
df_train_work = df_train_work.assign(img_alt='', title='', h1='', h2='', h3='', strong='', bold='', lang_code='', figcaption='')

In [103]:
# Befüllen der Extra-Features

for i in range (4, 13):
  for j in range(0, len(df_train_work)):
        if i == 4:
          df_train_work.iloc[:, i][j] = getImgDescriptionHTMLtag(df_train_work.html[j])
        elif i == 5:
          df_train_work.iloc[:, i][j] = getTitleHTMLtag(df_train_work.html[j])
        elif i == 6:
          df_train_work.iloc[:, i][j] = getH1HTMLtag(df_train_work.html[j])
        elif i == 7:
          df_train_work.iloc[:, i][j] = getH2HTMLtag(df_train_work.html[j])
        elif i == 8:
          df_train_work.iloc[:, i][j] = getH3HTMLtag(df_train_work.html[j])
        elif i == 9:
          df_train_work.iloc[:, i][j] = getStrongHTMLtag(df_train_work.html[j])
        elif i == 10:
          df_train_work.iloc[:, i][j] = getBoldHTMLtag(df_train_work.html[j])
        elif i == 11:
          df_train_work.iloc[:, i][j] = getLangHTMLtag(df_train_work.html[j])
        elif i == 12:
          df_train_work.iloc[:, i][j] = getFigCaptionHTMLtag(df_train_work.html[j])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launc

In [106]:
# aus List-Elementen in DataFrame einfache Strings machen, um besser cleanen zu können

# img
for x in range(len(df_train_work)):
    df_train_work.img_alt[x] = ' '.join(df_train_work.img_alt[x])

# h1
for x in range(len(df_train_work)):
    df_train_work.h1[x] = ' '.join(df_train_work.h1[x])

# h2
for x in range(len(df_train_work)):
    df_train_work.h2[x] = ' '.join(df_train_work.h2[x])

# h3
for x in range(len(df_train_work)):
    df_train_work.h3[x] = ' '.join(df_train_work.h3[x])

# strong
for x in range(len(df_train_work)):
    df_train_work.strong[x] = ' '.join(df_train_work.strong[x])

# bold
for x in range(len(df_train_work)):
    df_train_work.bold[x] = ' '.join(df_train_work.bold[x])

# figcaption
for x in range(len(df_train_work)):
    df_train_work.figcaption[x] = ' '.join(df_train_work.figcaption[x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykerne

In [107]:
# alle extra-features durchgehen
columns = df_train_work.columns.tolist()
columns = columns[4:13]

In [111]:
# alle extra-features durch clean_text schicken
for x in columns:
    for y in range(len(df_train_work)):
        df_train_work[x][y] = clean_text(df_train_work[x][y])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# Datei(en) exportieren

In [ ]:
df_train_work.to_json('train_preprocessed.json')

In [126]:
## wieder einlesen:

#df = pd.read_json('File Name.json')